# https://towardsdatascience.com/ai-agents-from-zero-to-hero-part-1/

# 0 - Setup

In [ ]:
#pip install ollama==0.4.7
import ollama
llm = "llama3.2"

In [16]:
stream = ollama.generate(model=llm, prompt='''what time is it?''', stream=True)
for chunk in stream:
    print(chunk['response'], end='', flush=True)

ResponseError: model requires more system memory (5.6 GiB) than is available (4.9 GiB) (status code: 500)

# 1 - Single Agent

## 1.1 - Simple (WebSearch)

In [3]:
#pip install langchain-community==0.3.17
#pip install duckduckgo_search==6.3.5
from langchain_community.tools import DuckDuckGoSearchResults 

def search_web(query: str) -> str:
  return DuckDuckGoSearchResults(backend="news").run(query)

tool_search_web = {'type':'function', 'function':{
  'name': 'search_web',
  'description': 'Search the web',
  'parameters': {'type': 'object', 
                 'required': ['query'],
                 'properties': {
                    'query': {'type':'str', 'description':'the topic or subject to search on the web'},
}}}}
## test
search_web(query="lookman gasperini")

"snippet: There was a feeling of deja vu among Nigerian fans this week. After Atalanta were dumped out of the Champions League by Club Brugge, their manager Gian Piero Gasperini ripped into Ademola Lookman. The Serie A side came into the second leg of the playoff tie trailing 2-1., title: Lookman vs Gasperini: A close look at treatment of Nigerian footballers in Serie A, link: https://www.msn.com/en-us/sports/other/lookman-vs-gasperini-a-close-look-at-treatment-of-nigerian-footballers-in-serie-a/ar-AA1zysFZ, date: 2025-02-22T06:09:00+00:00, source: MSN, snippet: Former Liverpool defender Steve Nicol has exonerated Ademola Lookman and named three persons to blame after the penalty incident against Club Brugge., title: Ademola Lookman: Ex Liverpool Star Names 3 People to Blame After Penalty Incident, link: https://www.msn.com/en-xl/africa/nigeria/ex-liverpool-star-reveals-3-people-to-blame-after-lookmans-penalty-incident/ar-AA1zyXpO, date: 2025-02-22T09:08:00+00:00, source: Legit, snippe

## Passaggio 5: Creazione del Tool per Ollama

```python
tool_search_web = {'type':'function', 'function':{
  'name': 'search_web',
  'description': 'Search the web',
  'parameters': {
    'type': 'object',
    'required': ['query'],
    'properties': {
      'query': {'type':'str', 'description':'the topic or subject to search on the web'}
    }
  }
}}
```

### 🔹 Cos'è questo dizionario?
Definisce un **Tool** in un formato che Ollama può comprendere.  

### 🔹 Perché serve?
Ollama **non può eseguire direttamente funzioni Python**, quindi dobbiamo descrivergli le funzioni disponibili in formato JSON.

---

## 💡 Analizziamolo riga per riga

### 1. Definizione del tipo di Tool
```python
{'type':'function', 'function':{
```
* Specifica che questo è un **Tool di tipo "function"**.

### 2. Nome della funzione
```python
'name': 'search_web',
```
* Nome della funzione (sarà riconosciuto da Ollama).

### 3. Descrizione della funzione
```python
'description': 'Search the web',
```
* Descrizione della funzione (Ollama userà questa informazione per capire cosa fa).

### 4. Definizione dei parametri
```python
'parameters': {'type': 'object',
```
* Definisce i **parametri** richiesti dalla funzione.

### 5. Parametri obbligatori
```python
'required': ['query'],
```
* Specifica che **l’unico parametro obbligatorio è "query"**.

### 6. Definizione delle proprietà dei parametri
```python
'properties': {
  'query': {'type':'str', 'description':'the topic or subject to search on the web'}
}
```
* **Definisce il parametro "query"**:
  * **`query`**: deve essere una **stringa**.
  * **Descrizione**: spiega che serve a specificare il **soggetto della ricerca**.

---

### 💡 A cosa serve questo dizionario?
👉 **Ollama userà queste informazioni per capire come chiamare la funzione.**  
Se gli chiedi qualcosa come:  
_"Find the latest news about Apple"_  
potrà usare il Tool per cercare su internet.

In [4]:
def search_yf(query: str) -> str:
  return DuckDuckGoSearchResults(backend="news").run(f"site:finance.yahoo.com {query}")

tool_search_yf = {'type':'function', 'function':{
  'name': 'search_yf',
  'description': 'Search for specific financial news',
  'parameters': {'type': 'object', 
                 'required': ['query'],
                 'properties': {
                    'query': {'type':'str', 'description':'the financial topic or subject to search'},
}}}}
## test
search_yf(query="nvidia")

'snippet: Nvidia (NASDAQ: NVDA) has been one of the best-performing stocks on the planet in recent years. The company has created an artificial intelligence (AI) empire, making itself the "go to" player for any customer aiming to build an AI platform., title: Is Nvidia a Bargain Buy Before Feb. 26? The Evidence Is Piling Up and Here\'s What It Shows., link: https://finance.yahoo.com/news/nvidia-bargain-buy-feb-26-091000850.html, date: 2025-02-22T09:10:00+00:00, source: YAHOO!Finance, snippet: Nvidia options are somewhat expensive heading into the company\'s earnings next week. - . Tech bellwether Nvidia NVDA is due to report earnings on Feb. 26 after the c, title: Nvidia options traders are vastly overestimating how earnings will move the stock. Just look at these charts., link: https://finance.yahoo.com/news/nvidia-options-traders-vastly-overestimating-220900521.html?fr=sycsrp_catchall, date: 2025-02-21T18:17:00+00:00, source: YAHOO!Finance, snippet: Market Domination Overtime hosts J

In [ ]:
prompt = '''You are an AI assistant. You should answer user questions directly whenever possible. 
Use tools only if the user asks for specific information that requires an external search.'''
messages = [{"role":"system", "content":prompt}]

while True:
    ## user input
    try:
        q = input('🙂 >')
        print(f"DEBUG: Hai scritto -> {q}")  # Debug
    except EOFError:
        break
    if q == "quit":
        break
    if q.strip() == "":
        continue
    messages.append( {"role":"user", "content":q} )
    
    print("DEBUG: Sto chiamando ollama.chat...")
    agent_res = ollama.chat(
        model=llm,
        stream=False,
        tools=[tool_search_web, tool_search_yf],
        messages=messages
    )
    print("DEBUG: ollama.chat() ha risposto!")
    print("DEBUG: Risultato da ollama.chat():", agent_res)

#############
??????????????????????????????
#############

DEBUG: Hai scritto -> Enter
DEBUG: Sto chiamando ollama.chat...
DEBUG: ollama.chat() ha risposto!
DEBUG: Risultato da ollama.chat(): model='llama3.2' created_at='2025-02-22T09:54:18.018084Z' done=True done_reason='stop' total_duration=24464067300 load_duration=66746800 prompt_eval_count=241 prompt_eval_duration=24168000000 eval_count=2 eval_duration=222000000 message=Message(role='assistant', content='None', images=None, tool_calls=None)
DEBUG: Hai scritto -> quit
